In [0]:
# Gold スキーマ設定
catalog_name = "users"
schema = "yukiteru_koide"

# スキーマ切り替え
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema}")

In [0]:
# Silver テーブル読み込み
df = spark.table(f"{catalog_name}.{schema}.transactions_enriched")

from pyspark.sql.functions import year, month, sum as _sum, avg

monthly_sales = (
    df
    .withColumn("year", year("transaction_date"))
    .withColumn("month", month("transaction_date"))
    .groupBy("year", "month", "store_id", "store_name", "category")
    .agg(
        _sum("total_price").alias("monthly_sales"),
        _sum("quantity").alias("monthly_quantity"),
        avg("price").alias("avg_unit_price")
    )
)

monthly_sales.write.format("delta").mode("overwrite").saveAsTable("monthly_sales_by_store_and_category")
